# LSTM TRACKER

In [ ]:
# install required dependencies
!pip install yfinance

In [ ]:
# Load necessary liraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
SEED = 42

## Introduction

In project i will demonstrate the application of LSTM model in predicting the stock price.

In [ ]:
# let's load our stock data
# here, i will use the stock data of TESLA

tesla_ticker = yf.Ticker('TSLA') # extracted the ticker data 
# Load the full historical stock data of Tesla 
tesla_stc = tesla_ticker.history(period = "10y").reset_index()

In [ ]:
tesla_stc.head()

In [ ]:
tesla_stc.info()

In [ ]:
tesla_stc.describe()

## EDA

In [ ]:
print(f"Tesla Stock LTP : ${round(tesla_stc.Close.iloc[-1], 2)}")
print(f"Tesla Stock Current Volume : {round(tesla_stc.Volume.iloc[-1], 2)}")

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (18, 5))

# Open Price Growth
sns.lineplot(data = tesla_stc, x= tesla_stc['Date'], y= tesla_stc['Open'], c = 'green', ax = ax[0])
ax[0].set_ylabel('Open', labelpad = 20)
ax[0].set_xlabel('Date', labelpad = 20)


# Close Price Growth
sns.lineplot(data = tesla_stc, x= tesla_stc['Date'], y= tesla_stc['Close'], c = 'red', ax = ax[1])
ax[1].set_ylabel('Close', labelpad = 20)
ax[1].set_xlabel('Date', labelpad = 20)

In [ ]:
# Tesla Stock Growth in 5 Years
tsla_five_year_growth = tesla_stc.loc[tesla_stc['Date'] >= '2019-06-01']
fig = plt.figure(figsize = (11,5))
sns.lineplot(data=tsla_five_year_growth, x ='Date', y='Close', c= 'green')
plt.title('Tesla Stock Growth in 5 Years')
plt.xlabel('Date', labelpad = 20)
plt.ylabel('Close', labelpad = 20)
plt.show()

# Feature Selection

In [ ]:
# dropping stock split and dividends columns
tesla_stc_train = tesla_stc.drop(columns = ['Open','Stock Splits', 'Dividends', 'Low', 'High','Volume'], axis =1 )
tesla_stc_train.head()

## Feature Engineering

In [ ]:
 tesla_stc_train.Date = pd.to_datetime(tesla_stc_train.Date.dt.strftime("%Y-%m-%d"))

In [ ]:
tesla_stc_train.set_index('Date', inplace = True)

In [ ]:
tesla_stc_train.head()

# Normalization

In [ ]:
# Normalizing the stock data 
scaler = MinMaxScaler()
tesla_stc_train[tesla_stc_train.columns] = scaler.fit_transform(tesla_stc_train)

In [ ]:
tesla_stc_train.shape

In [ ]:
# spltting the data into train and test sets
train_size = int(0.65 * len(tesla_stc_train))
train_data = tesla_stc_train.iloc[:train_size, :]
test_data = tesla_stc_train.iloc[train_size:,:]

In [ ]:
train_data.shape, test_data.shape

In [ ]:
def create_sequences(data, n_steps):
    sequences = []
    labels = []
    for indx in range(len(data)-n_steps-1):
        sequences.append(data[indx: (indx+n_steps) ].values)
        labels.append(data.iloc[indx + n_steps, 0])
    return np.array(sequences), np.array(labels)

In [ ]:
train_x, train_y = create_sequences(train_data, 100)
test_x, test_y = create_sequences(test_data, 100)

In [ ]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

# LSTM

In [ ]:
# create sequential model
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(train_x.shape[0], train_x.shape[2])))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_x, train_y, epochs = 100, validation_data=(test_x, test_y), batch_size=64, verbose=1)

In [ ]:
train_predicted = model.predict(train_x)
test_predicted = model.predict(test_x)

In [ ]:
train_predicted = scaler.inverse_transform(train_predicted)
test_predicted = scaler.inverse_transform(test_predicted)

In [ ]:
tesla_stc_train[tesla_stc_train.columns] = scaler.inverse_transform(tesla_stc_train)

In [ ]:
test_predicted.shape

In [ ]:
tesla_stc_predicted = pd.concat([tesla_stc_train.iloc[-781:,0].copy(),pd.DataFrame(test_predicted,columns=['Close_Predicted'],index=tesla_stc_train.iloc[-781:,0].index)], axis=1)

In [ ]:
tesla_stc_predicted

In [ ]:
tesla_stc_predicted[['Close','Close_Predicted']].plot(figsize=(10,6))
plt.xticks(rotation=45)
plt.xlabel('Date',size=15)
plt.ylabel('Stock Price',size=15)
plt.title('Actual vs Predicted for open price',size=15)
plt.show()